In [ ]:
from tensorflow._api.v2.image import decode_png, convert_image_dtype, resize, random_flip_left_right, random_flip_up_down
from tensorflow._api.v2.strings import split, to_number
from tensorflow._api.v2.data import Dataset, AUTOTUNE
from tensorflow._api.v2.io import read_file
from tuneable_model import CancerNet
from keras.utils import to_categorical
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.optimizers import Adagrad
import tensorflow as tf 
from imutils.paths import list_images
from config import *
import os
import keras_tuner as kt


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
def load_images(path):
    image = read_file(path)
    image = decode_png(image)
    image = convert_image_dtype(image, "float32")
    image = resize(image, (48, 48))

    label = split(path, os.path.sep)[-2]
    label = to_number(label, tf.int32)
    return image, label


In [5]:
@tf.function
def augment(image, label):
    image = random_flip_left_right(image)
    image = random_flip_up_down(image)
    return image, label

In [6]:
trainPaths = list(list_images(TRAIN_PATH))
testPaths = list(list_images(TEST_PATH))
valPaths = list(list_images(VAL_PATH))

In [7]:
trainLabel = [path.split(os.path.sep)[-2] for path in trainPaths]
testLabel= [path.split(os.path.sep)[-2] for path in testPaths]
valLabel= [path.split(os.path.sep)[-2] for path in valPaths]


In [8]:
onehot = to_categorical(trainLabel)
classTotal = onehot.sum(axis=0)
classWeights = {}

In [9]:
for i in range(len(classTotal)):
    classWeights[i] = classTotal.max() / classTotal[i]

In [10]:
trainDS = Dataset.from_tensor_slices(trainPaths)
testDS = Dataset.from_tensor_slices(testPaths)
valDS = Dataset.from_tensor_slices(valPaths)

In [11]:
trainDS = (trainDS
           .shuffle(len(trainPaths))
           .map(load_images, num_parallel_calls=AUTOTUNE)
           .map(augment, num_parallel_calls=AUTOTUNE)
           .cache()
           .batch(BS)
           .prefetch(AUTOTUNE))
testDS = (testDS
          .map(load_images, num_parallel_calls=AUTOTUNE)
          .cache()
          .batch(BS)
          .prefetch(AUTOTUNE))
valDS = (valDS
          .map(load_images, num_parallel_calls=AUTOTUNE)
          .cache()
          .batch(BS)
          .prefetch(AUTOTUNE))

In [15]:
tuner = kt.Hyperband(
    CancerNet.build,
    objective="val_accuracy",
    max_epochs=NUM_EPOCHS,
    factor=3,
    seed=42,
    directory=OUTPUT_PATH,
    project_name="hyperband"
)

In [17]:
es = EarlyStopping(
            monitor="val_loss",
            patience=5
        )

In [ ]:
tuner.search(
    trainDS,
    validation_data=valDS,
    batch_size=BS,
    callbacks=[es],
    epochs=NUM_EPOCHS
)

In [19]:
bestHP = tuner.get_best_hyperparameters()[0]
model = tuner.hypermodel.build(bestHP)

In [ ]:
bestHP

In [20]:
model.fit(
    trainDS,
    validation_data=valDS,
    batch_size=BS,
    epochs=NUM_EPOCHS,
    callbacks=[es],
    class_weight=classWeights,
    verbose=1
)

Epoch 1/250
2169/2169 [==============================] - 610s 281ms/step - loss: 0.0000e+00 - accuracy: 0.8425 - val_loss: 0.0000e+00 - val_accuracy: 0.8113
Epoch 2/250
2169/2169 [==============================] - 557s 257ms/step - loss: 0.0000e+00 - accuracy: 0.8598 - val_loss: 0.0000e+00 - val_accuracy: 0.8561
Epoch 3/250
2169/2169 [==============================] - 574s 265ms/step - loss: 0.0000e+00 - accuracy: 0.8672 - val_loss: 0.0000e+00 - val_accuracy: 0.8673
Epoch 4/250
2169/2169 [==============================] - 3300s 2s/step - loss: 0.0000e+00 - accuracy: 0.8705 - val_loss: 0.0000e+00 - val_accuracy: 0.8691
Epoch 5/250
2169/2169 [==============================] - 621s 286ms/step - loss: 0.0000e+00 - accuracy: 0.8718 - val_loss: 0.0000e+00 - val_accuracy: 0.8746
Epoch 6/250
2169/2169 [==============================] - 1643s 758ms/step - loss: 0.0000e+00 - accuracy: 0.8735 - val_loss: 0.0000e+00 - val_accuracy: 0.8774


In [23]:
loss, accuracy = model.evaluate(testDS)

1215/1215 [==============================] - 71s 58ms/step - loss: 0.0000e+00 - accuracy: 0.8761


[0.0, 0.8761115670204163]

In [24]:
print("[INFO] accuracy: {:.3f}%".format(accuracy * 100))